### Загружаем данные

* Данные по кампаниям за 1 год
* **165823** кампаний
* **29** параметров
* Параметры рассчитывались за **0,1,2 и 3 дни** жизни пользователя
* Нужно прогнозировать **CARPU 60 дня**

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('data_users.csv',delimiter =',')
data.head()

,user,y_target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
0,user01,17.581401,0.0000,3.5161,0.0000,14.065301,36,5307.0,3.0,1769.0,...,1.0,0.917808,0.917808,0.026667,0.026667,1.000000,1.000000,0.032520,0.032520,27.0
1,user02,3.825400,3.8254,0.0000,0.0000,0.000000,8,101.0,1.0,101.0,...,NaN,0.920000,0.920000,0.038462,0.038462,1.000000,1.000000,NaN,NaN,19.0
2,user03,94.554604,0.0000,0.0000,0.0000,94.554604,129,792.0,3.0,264.0,...,18.0,0.868852,0.868852,0.197368,0.197368,0.933333,0.933333,0.151899,0.151899,31.0
3,user04,15.598800,0.0000,9.0994,6.4994,0.000000,108,390.0,1.0,390.0,...,NaN,0.903226,0.903226,0.144828,0.144828,1.000000,1.000000,NaN,NaN,21.0
4,user05,3.051300,3.0513,0.0000,0.0000,0.000000,132,NaN,NaN,NaN,...,2.0,0.909091,0.909091,0.297872,0.297872,1.000000,1.000000,0.222222,0.222222,21.0


In [2]:
data.describe()

,y_target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
count,165823.000000,165823.000000,165823.000000,165823.000000,165823.000000,165823.000000,1.207870e+05,120787.000000,120787.000000,164560.000000,...,106375.000000,164496.000000,164496.000000,146068.000000,146068.000000,143014.000000,143014.000000,105929.000000,105929.000000,164643.000000
mean,22.309657,7.973157,5.951624,4.142034,3.165667,46.105160,1.856913e+03,2.254978,659.612587,44.756849,...,4.396489,0.902721,0.902721,0.225143,0.225143,0.946322,0.946322,0.244371,0.244371,23.091276
std,103.533190,27.890069,31.351652,34.962670,28.377308,137.739549,1.793705e+04,4.147049,4632.573640,36.558349,...,5.076930,0.088137,0.088137,0.122492,0.122492,0.091679,0.091679,0.179817,0.179817,8.656326
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,...,0.166667,0.058824,0.058824,0.002445,0.002445,0.100000,0.100000,0.003077,0.003077,2.000000
25%,3.480200,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000e+02,1.000000,50.330000,25.000000,...,1.500000,0.854545,0.854545,0.125000,0.125000,0.916667,0.916667,0.108696,0.108696,18.000000
50%,6.127000,3.068700,0.000000,0.000000,0.000000,22.000000,1.800000e+02,2.000000,103.000000,36.000000,...,3.000000,0.918919,0.918919,0.240000,0.240000,1.000000,1.000000,0.200000,0.200000,24.000000
75%,14.951150,7.198850,3.740000,0.000000,0.000000,48.000000,6.000000e+02,3.000000,253.000000,53.000000,...,5.750000,0.968750,0.968750,0.309859,0.309859,1.000000,1.000000,0.333333,0.333333,30.000000
max,18788.989770,2663.212256,4390.317388,7691.737705,6117.921587,19426.000000,3.511290e+06,264.000000,877822.500000,1178.000000,...,270.750000,1.000000,1.000000,0.972973,0.972973,1.000000,1.000000,0.992084,0.992084,76.000000


### Поработаем с данными
* Заполним NaN
* Нормализуем значения параметров
* Промасштабируем значения параметров

In [3]:
#Сколько у нас пропущенных значений?
missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head(20)

,Missing Ratio
feature_27,36.119236
feature_28,36.119236
feature_20,35.850274
feature_19,35.850274
feature_6,27.159079
feature_7,27.159079
feature_8,27.159079
feature_26,13.755028
feature_25,13.755028
feature_16,13.755028


In [4]:
# Заполним пропущенные значения и проверим
data.fillna(0, inplace = True)

missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head()

,Missing Ratio


In [ ]:
# Зададим X и y
X = data.iloc[:,2:31]
y = data['y_target']
y.describe()

In [ ]:
# Нормализуем данные и смотрим коэффициент ассиметрии
from scipy.special import boxcox1p
from scipy.stats import skew
import numpy as np

numeric_feats = X.dtypes[X.dtypes!="object"].index
skewed_feats = X[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame({'Skew':skewed_feats})
skewness = skewness[abs(skewness)>0.2]
skewed_features = skewness.index
lam=0.04
for feat in skewed_features:
    X[feat] = boxcox1p(X[feat],lam)

# Логарифмируем target и смотрим коэффициент ассиметрии
y_n=boxcox1p(y,0)
print("Коэффициент ассиметрии y до транфсормации – %s и после – %s"% (skew(y),skew(y_n)))

In [ ]:
y_n.describe()

In [ ]:
# Масштабируем с Robust Scaler-ом
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()
names = list(X.columns)
X= sc.fit_transform(X)
X = pd.DataFrame(data=X, columns=names)
X.fillna(0, inplace = True)

In [ ]:
X.head()

### Перейдем к моделированию
* **Модель:** Будем использовать Ridge regression
* **Кросс-валидация**: алгоритм ShuffleSplit
* **Работа с параметрами:** Pearson score, RFECV
* **Регуляризация**. Посмотрим как выбирается параметр Alpha
* **Learning Curve**: Посмотрим как ведет себя модель на обучающей и валидирующей выборках
* **Residuals Plot**: Посмотрим как ведут себя остатки (разница между реальным и пронозным значениями)
* **Prediction Error Plot**: Сравним прогнозные результаты модели с ее реальными значениями

In [ ]:
from sklearn.linear_model import  RidgeCV
from sklearn.model_selection import ShuffleSplit, train_test_split

cv=ShuffleSplit(n_splits=4,train_size=0.8,test_size=0.2,random_state= 2019)
regressor = RidgeCV()

### Смотрим R2 и MSE

In [ ]:
import matplotlib.pyplot as plt
from yellowbrick.model_selection import CVScores

_, ax = plt.subplots(figsize=(10,10))
r2 = CVScores(regressor,ax=ax,cv=cv,scoring='r2')
r2.fit(X,y_n)
r2.poof()

In [ ]:
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring='neg_mean_squared_error')
mse.fit(X,y_n)
mse.poof()

In [ ]:
from sklearn.metrics import make_scorer

def rmse(y, y_pred):
    return np.sqrt(np.mean((y_pred - y)**2))
scorer = make_scorer(rmse,greater_is_better=False)
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring=scorer)
mse.fit(X,y_n)
mse.poof()

In [ ]:
y_n.describe()

### Поработаем с параметрами

In [ ]:
# Посмотрим коллеряционную матрицу
from yellowbrick.features import Rank2D,RFECV
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=names,algorithm='pearson')
rank.fit(X,y_n)
rank.transform(X)
rank.poof()

In [ ]:
%%time
# Попробуем посмотреть Recursive Feature Elimination
_, ax = plt.subplots(figsize=(20,10))
rfe = RFECV(regressor,cv=cv,scoring=scorer,step=15)
rfe.fit(X,y_n)
rfe.poof()

In [ ]:
#Сохраним выбранные параметры
params = {
    'parameters': list(X.columns),
    'ranking':list(rfe.ranking_)
}
par = pd.DataFrame(data=params)
huber_params = list(par[par['ranking']==1]['parameters'])

In [ ]:
# Посмотрим коллеряционную матрицу еще раз
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=huber_params,algorithm='pearson')
rank.fit(X[huber_params],y_n)
rank.transform(X[huber_params])
rank.poof()

### Теперь поработаем с регуляризационным коэффициентом alpha

In [ ]:
from yellowbrick.regressor import AlphaSelection
_, ax = plt.subplots(figsize=(20,10))
alphas = np.logspace(-10, 1, 400)
a_select=AlphaSelection(RidgeCV(alphas=alphas))
a_select.fit(X[huber_params],y_n)
a_select.poof()

### Посмотрим Learning Curve

In [ ]:
%%time
from yellowbrick.model_selection import LearningCurve
sizes = np.linspace(0.2, 1.0, 500)
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring=scorer,cv=cv)
lc.fit(X[huber_params],y_n)
lc.poof()

In [ ]:
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring='r2',cv=cv)
lc.fit(X[huber_params],y_n)
lc.poof()

### Посмотрим, что нам дает обученная модель

In [ ]:
# Смотрим как ведут себя остатки
from sklearn.model_selection import train_test_split
from yellowbrick.regressor import ResidualsPlot,PredictionError
X_train,X_test,y_train,y_test = train_test_split(X[huber_params],y_n,test_size=0.2,random_state=2019)
regressor=RidgeCV(alphas=alphas,cv=cv,scoring=scorer)

_, ax = plt.subplots(figsize=(20,10))
res = ResidualsPlot(regressor)
res.fit(X_train,y_train)
res.score(X_test,y_test)
res.poof()

In [ ]:
# Смотрим как распределены ошибки
_, ax = plt.subplots(figsize=(20,10))
error=PredictionError(regressor)
error.fit(X_train,y_train)
error.score(X_test,y_test)
error.poof()